In [ ]:
# import splunklib as splunk
import splunklib.client as client
import splunklib.results as results
import urllib.parse
from pandas.io.sql import SQLTable

def _execute_insert(self, conn, keys, data_iter):
    data = [dict(zip(keys, row)) for row in data_iter]
    conn.execute(self.table.insert().values(data))

SQLTable._execute_insert = _execute_insert

import pandas as pd
import vertica_python
import sqlalchemy as sa
import pyodbc
import sys
from sqlalchemy import create_engine
from sqlalchemy import inspect
from sqlalchemy.types import VARCHAR
from sqlalchemy.types import Integer
from time import sleep
from pandas.io.sql import SQLTable
import psycopg2
import io #
import tableauserverclient as TSC

print("Connect to Splunk...\n")

HOST = "host"    #connection param
PORT = 8089
USERNAME = "username" # your username
PASSWORD = 'password' # your password
APP = "APP"

service = client.connect(
    host=HOST,
    port=PORT,
    username=USERNAME,
    password=PASSWORD,
    app=APP)


print("Running Splunk query...\n")


mysavedsearch = service.saved_searches["data_dashboards"] #My Dashboard
job = mysavedsearch.dispatch()


while True:
    while not job.is_ready():
        pass
    stats = {"isDone": job["isDone"],
             "doneProgress": float(job["doneProgress"])*100,
              "scanCount": int(job["scanCount"]),
              "eventCount": int(job["eventCount"]),
              "resultCount": int(job["resultCount"])}

    status = ("\r%(doneProgress)03.1f%%   %(scanCount)d scanned   "
              "%(eventCount)d matched   %(resultCount)d results") % stats

    sys.stdout.write(status)
    sys.stdout.flush()
    if stats["isDone"] == "1":
        sys.stdout.write("\n Splunk query Done!\n\n")
        break
    sleep(2)

# Create a Dataframe
d = []            
for result in results.ResultsReader(job.results(count = 0)):
    d.append(result)

job.cancel()   
sys.stdout.write('\n')

df = pd.DataFrame(d)  # main df
print("DataFrame Ready...\n")

print("Field astype config\n")

df['Field_1']= df['Field_1'].astype(int) #KEY
df['Field_2']= df['Field_2'].astype(int)
df['Field_3']= df['Field_3'].astype(int)
df['Field_4_date']= pd.to_datetime(df['Field_4_date'])
df['Field_5']= df['Field_5'].astype(float)


con1 = vertica_python.connect(host='host_vertica', 
                              port=5433, 
                              dbname='dbname', 
                              user='user', 
                              password='password')
cursor = con1.cursor()

cursor.execute("""truncate table shema.table_temp;""")
copy_str = """COPY shema.table_temp(
Field_1,
Field_2,
Field_3,
Field_4_date,
Field_5
)FROM STDIN DELIMITER ','"""

stream = io.StringIO()
df.to_csv(stream, sep=",",index=False, header=False)

print("Stream init...\n")
stream.seek(0) 


with con1.cursor() as cursor:
    cursor.copy(copy_str,stream.getvalue())
con1.commit()

print("shema.table_temp ...ok\n")

cursor.execute("""select count(case when a.Field_1 is null then 1 else null end) new_record,
        count(case when b.Field_1 is null then 1 else null end) archived_record,
        count(case when a.Field_1 is not null and b.Field_1 is not null then 1 else null end) overlap,
        count(*) total_records
from shema.table_main a full outer join
        shema.table_temp b on a.Field_1 = b.Field_1;""")


q1row = cursor.fetchall()
print('new_record - archived_record - overlap - total_records')
print(q1row)


print('\n')

cursor.execute("""drop table shema.table_backup;""")
#Drop if right OK

cursor.execute("""select *
into shema.table_backup
from shema.table_main a
;""")
cursor.execute("""delete from shema.table_main where TICKET_ID in 
(
select distinct a.Field_1
from shema.table_main a full outer join
        shema.table_temp b on a.Field_1 = b.Field_1
where a.Field_1 is not null and b.Field_1 is not null
);""")
cursor.execute("""insert into shema.table_main
select *
from shema.table_temp
;""")
con1.commit()

print("shema.table_backup ...ok\n")
print("shema.table_main ...ok\n")


print("DONE :)\n")

print('Refresh Tableau Server...\n') #tableau update


tableau_auth = TSC.TableauAuth('user', 'password', site_id = 'site_id')
server = TSC.Server('host_tbleau',use_server_version = True)
server.auth.sign_in(tableau_auth)
print('connection made\n')
print(server.version)
server.workbooks.refresh(workbook_id='xxxxxxxxxxxx')
server.workbooks.refresh(workbook_id='xxxxxxxxxxxx')
server.workbooks.refresh(workbook_id='xxxxxxxxxxxx')
server.workbooks.refresh(workbook_id='xxxxxxxxxxxx')
print('refresh complete\n')

server.auth.sign_out()
print('connection closed\n')



print("DONE :)")

